In [81]:
from keras import models
from keras import layers
from keras import activations
from keras import initializers
from keras import regularizers
from keras import optimizers
from keras import losses
from keras import metrics
from keras import callbacks
from keras.preprocessing import image

In [82]:
model = models.Sequential()
model.add(layers.Conv2D(
    32, (5, 5),
    input_shape = (64, 64, 3) # x = 60 * 32
))
model.add(layers.MaxPooling2D(
    pool_size = (2, 2)
)) # x = x / 2
model.add(layers.Conv2D(
    16, (5, 5),
    input_shape = (30, 30, 3)
)) # x = (x - 4) * 16
model.add(layers.MaxPooling2D(
    pool_size = (2, 2)
)) # x = x / 2
# x = x^2
model.add(layers.Flatten())
model.add(layers.Dense(512, activation="relu"))

model.add(layers.Dense(128, activation="relu"))
model.add(layers.Dropout(0.2))

model.add(layers.Dense(128, activation="relu"))

model.add(layers.Dense(128, activation="relu"))

model.add(layers.Dense(128, activation="relu"))

model.add(layers.Dense(30, activation="softmax"))

In [83]:
model.compile(
    optimizer=optimizers.Adam(),
    loss=losses.BinaryCrossentropy(),
    metrics=[metrics.CategoricalAccuracy(), metrics.Precision()]
)

In [84]:
dataGen = image.ImageDataGenerator(
    rescale = 1.0/255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    vertical_flip = False,

    validation_split = 0.2
)

X_train = dataGen.flow_from_directory(
    'Images',
    target_size = (64, 64),
    batch_size = 32,
    class_mode = 'categorical',
    subset = 'training'
)

X_tests = dataGen.flow_from_directory(
    'Images',
    target_size = (64, 64),
    batch_size = 32,
    class_mode = 'categorical',
    subset = 'validation'
)

Found 24000 images belonging to 30 classes.
Found 6000 images belonging to 30 classes.


In [85]:
total_data = 24000
batchSize = 32

model.fit(X_train,
          validation_data = X_tests,
          steps_per_epoch = total_data / batchSize,
          epochs = 50,
          validation_steps = (total_data / batchSize) / 10,
          callbacks = [
            callbacks.ModelCheckpoint(
                filepath = 'ModelCheckpoint/model.{epoch:02d}-{val_loss:.2f}.h5'
            )
          ]
)

model.save('model')

Epoch 1/50
750/750 [==============================] - 63s 83ms/step - loss: 0.1369 - categorical_accuracy: 0.1314 - precision_16: 0.3721 - val_loss: 0.1276 - val_categorical_accuracy: 0.1858 - val_precision_16: 0.5435
Epoch 2/50
750/750 [==============================] - 64s 85ms/step - loss: 0.1171 - categorical_accuracy: 0.2412 - precision_16: 0.5102 - val_loss: 0.1180 - val_categorical_accuracy: 0.2338 - val_precision_16: 0.4119
Epoch 3/50
750/750 [==============================] - 62s 83ms/step - loss: 0.1096 - categorical_accuracy: 0.2963 - precision_16: 0.5605 - val_loss: 0.1162 - val_categorical_accuracy: 0.2654 - val_precision_16: 0.5153
Epoch 4/50
750/750 [==============================] - 63s 84ms/step - loss: 0.1036 - categorical_accuracy: 0.3371 - precision_16: 0.5935 - val_loss: 0.1129 - val_categorical_accuracy: 0.2875 - val_precision_16: 0.4664
Epoch 5/50
750/750 [==============================] - 63s 84ms/step - loss: 0.0982 - categorical_accuracy: 0.3745 - precision_16

FailedPreconditionError: model\variables is not a directory